In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

# Setup
dataset_root = './data'
num_epochs = 10
batch_size = 64
num_cls = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Data transformations and loading
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

trainset = datasets.CIFAR100(root=dataset_root, train=True, transform=data_transforms, download=True)
testset = datasets.CIFAR100(root=dataset_root, train=False, transform=data_transforms, download=True)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Model setup
model = models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_cls)
model = model.to(device)

# Loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Model training and evaluation
best_acc = 0.0
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = test_loader

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

print(f'Best val Acc: {best_acc:.4f}')

# Save and load the best model
torch.save(best_model_wts, 'best_model.pth')
model.load_state_dict(torch.load('best_model.pth'))

# Test the model
model.eval()
total, correct = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f'Test Accuracy: {100.0 * correct / total:.2f}%')


Using device: cuda:0


100%|██████████| 169M/169M [00:18<00:00, 9.16MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 180MB/s]


Epoch 1/10
train Loss: 2.6947 Acc: 0.3837
val Loss: 1.8999 Acc: 0.5364
Epoch 2/10
train Loss: 1.8543 Acc: 0.5141
val Loss: 1.6575 Acc: 0.5682
Epoch 3/10
train Loss: 1.6983 Acc: 0.5396
val Loss: 1.5503 Acc: 0.5825
Epoch 4/10
train Loss: 1.6180 Acc: 0.5550
val Loss: 1.4919 Acc: 0.5874
Epoch 5/10
train Loss: 1.5686 Acc: 0.5650
val Loss: 1.4550 Acc: 0.5972
Epoch 6/10
train Loss: 1.5272 Acc: 0.5749
val Loss: 1.4261 Acc: 0.6047
Epoch 7/10
train Loss: 1.5027 Acc: 0.5768
val Loss: 1.4082 Acc: 0.6052
Epoch 8/10
train Loss: 1.4762 Acc: 0.5852
val Loss: 1.3883 Acc: 0.6119
Epoch 9/10
train Loss: 1.4504 Acc: 0.5886
val Loss: 1.3715 Acc: 0.6144
Epoch 10/10
train Loss: 1.4295 Acc: 0.5939
val Loss: 1.3605 Acc: 0.6185
Best val Acc: 0.6185


<ipython-input-3-76ae272babb3>:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Accuracy: 62.03%
